In [11]:
from io import StringIO
import numpy as np
import polars as pl
import pandas as pd
from subprocess import call
from gtfparse import read_gtf

### Set parameters

In [12]:
exon_flank_nt = 5 # flanking nucleotides from the start and end of exons
number_of_threads = 4 # number of threads used in bcftools output compression
tag_str = 'Vineel_090823' # DNAnexus job tag

project_path = 'project-GGy3Bb0JqBj7zfxY8v4by61X:/'

# input vcf path (end with '/')
dx_vcf_path = project_path + "Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants,\ pVCF\ format\ -\ final\ release/"
# output vcf path (end with '/')
dx_vcf_out_path = project_path + "Tian_folder/Vineel_genes_test/"
# resource path (end with '/')
dx_resource_path = project_path + "GRCh38_resources/"
# difficult regions bed filename
diff_bed = 'GRCh38_alldifficultregions.bed.gz'
# reference genome filename
ref_genome = 'GRCh38_reference_genome.fa' # index filename is inferred

!dx mkdir -p {dx_vcf_out_path} # create gene.vcf output folder

### Helper functions

In [13]:
def get_overlapping_UKB_vcfs(df_gene, df_blk):
    vcf_prefix = 'ukb23157_'
    vcf_suffix = '_v1.vcf.gz'
    
    vcf_files = []
    for index, row in df_gene.iterrows():
        vcf_files = vcf_files + df_blk.loc[(df_blk['seqname'] == row['seqname']) & 
                                           # when the gene is completely contained inside the block
                                           (((df_blk['start_pos'] <= row['exon_flank_start']) & (df_blk['end_pos'] >= row['exon_flank_end'])) |
                                            # when the block is completely contained inside the gene
                                            ((df_blk['start_pos'] >= row['exon_flank_start']) & (df_blk['end_pos'] <= row['exon_flank_end'])) |
                                            # when the gene overlaps with the start of the block
                                            (df_blk['start_pos'].between(row['exon_flank_start'], row['exon_flank_end'])) |
                                            # when the gene overlaps with the end of the block
                                            (df_blk['end_pos'].between(row['exon_flank_start'], row['exon_flank_end']))),
                                           'chr_blk_str'].tolist()
    vcf_files = set(vcf_prefix + x + vcf_suffix for x in set(vcf_files))
    return(vcf_files)

### List of gene symbols as input

In [14]:
# genes = ["BRCA1", "BRCA2", "MSH2", "MSH6", "MLH1", "PMS2", "EPCAM"]
genes = ["LPA"]

# # to read from txt:
# with open("resources/my_gene_file.txt", "r") as gene_file:
#     genes = gene_file.readlines()
#     genes = [l.replace("\n", "") for l in genes]
#     print(genes)

### Load pVCF block coordinates

In [15]:
df_blk = pd.read_table("./resources/pvcf_blocks.txt", sep = '\t', names = ['ind', 'chr', 'blk', 'start_pos', 'end_pos'])
df_blk['seqname'] = 'chr' + df_blk['chr'].map(str)
df_blk.loc[df_blk['seqname'] == 'chr23', 'seqname'] = 'chrX'
df_blk.loc[df_blk['seqname'] == 'chr24', 'seqname'] = 'chrY'
df_blk['chr_blk_str'] = df_blk['seqname'].str.replace('chr', 'c') + '_b' + df_blk['blk'].map(str)
df_blk

,ind,chr,blk,start_pos,end_pos,seqname,chr_blk_str
0,1,1,0,1,1218130,chr1,c1_b0
1,2,1,1,1218131,1426969,chr1,c1_b1
2,3,1,2,1426970,1758871,chr1,c1_b2
3,4,1,3,1758872,2514221,chr1,c1_b3
4,5,1,4,2514222,3782130,chr1,c1_b4
...,...,...,...,...,...,...,...
972,973,23,20,135552245,141897932,chrX,cX_b20
973,974,23,21,141897933,152168662,chrX,cX_b21
974,975,23,22,152168663,153788223,chrX,cX_b22
975,976,23,23,153788224,156040895,chrX,cX_b23


### Load MANE transcript coordinates

In [16]:
df = pd.read_csv("./resources/MANE.GRCh38.v1.0.select_ensembl_genomic.csv.gz")
df = df.loc[(df['feature'] == 'exon') & (df['gene_name'].isin(genes))]
df = df[['seqname', 'start', 'end', 'gene_name']]
df['exon_flank_start'] = df['start'] - exon_flank_nt
df['exon_flank_end'] = df['end'] + exon_flank_nt
df['region'] = ((df['seqname'] + ':').str.cat(df['exon_flank_start'].astype(str)) + '-').str.cat(df['exon_flank_end'].astype(str))
df

,seqname,start,end,gene_name,exon_flank_start,exon_flank_end,region
174707,chr6,160664166,160664275,LPA,160664161,160664280,chr6:160664161-160664280
174710,chr6,160650338,160650497,LPA,160650333,160650502,chr6:160650333-160650502
174712,chr6,160646214,160646395,LPA,160646209,160646400,chr6:160646209-160646400
174714,chr6,160644847,160645006,LPA,160644842,160645011,chr6:160644842-160645011
174716,chr6,160640667,160640848,LPA,160640662,160640853,chr6:160640662-160640853
174718,chr6,160639300,160639459,LPA,160639295,160639464,chr6:160639295-160639464
174720,chr6,160635123,160635304,LPA,160635118,160635309,chr6:160635118-160635309
174722,chr6,160633753,160633912,LPA,160633748,160633917,chr6:160633748-160633917
174724,chr6,160629574,160629755,LPA,160629569,160629760,chr6:160629569-160629760
174726,chr6,160628207,160628366,LPA,160628202,160628371,chr6:160628202-160628371


### Run Swiss-army-knife on DNAnexus
- get region info for each gene from block file
- bcftools command for step 2 finished
- bcftools command for step 3 & 4 TBD
- list genes not included in MANE set

In [21]:
genes_not_found = []
genes_found = []
known_large_genes = ["DSP", "TSC2", "TTN", "NCOA3"]

for gene in genes:
    df_gene = df.loc[df['gene_name'] == gene]
    if df_gene.shape[0] > 0:
        genes_found.append(gene)
        
        vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
        vcf_str = ' '.join(vcf_files)
        region_str = ','.join(df_gene['region'].to_list())
        
        if ((len(vcf_files) > 1) or (gene in known_large_genes)):
            mem_level = "mem2_ssd1_v2_x16" # dynamically change memory level when submitting jobs on DNAnexus
        else:
            mem_level = "mem1_ssd1_v2_x4"
        
        # filtering variants
        bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
        bcftools_cmd2 = "bcftools view -Ou --max-alleles 5 -T ^" + diff_bed
        bcftools_cmd3 = "bcftools +fill-tags -Ou -- -t all"
        bcftools_cmd4 = "bcftools norm -Ou -m - -f " + ref_genome 
        bcftools_cmd5 = "bcftools view -Ov -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + "_variants.vcf.gz"
        
        # extract carriers
        bcftools_cmd6 = "mkdir -p carrier_out"
        bcftools_cmd7 = "bcftools query -i 'GT=\\\"RA\\\"|GT=\\\"AR\\\"|GT=\\\"AA\\\"' -f '%CHROM  %POS %REF %ALT %INFO/AF [%SAMPLE|]\n' " + gene + "_variants.vcf.gz > carrier_out/" + gene + ".ssv"
        
        # parsing command
        bcftools_command_a = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])
        bcftools_command_b = " && ".join([bcftools_cmd6, bcftools_cmd7])
        bcftools_command = bcftools_command_a + " && " + bcftools_command_b
        
        # parsing input
        dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'
        
        # final dx command
        dx_command = 'dx run swiss-army-knife --instance-type ' + mem_level + ' -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene
        !{dx_command}
    else:
        genes_not_found.append(gene)
            
print('Genes not found in MANE database:')
print(genes_not_found)

job-GYz2zx0JqBj903ZYYVf98YXb
Genes not found in MANE database:
[]


In [12]:
dx_command_c

'dx run swiss-army-knife --instance-type mem2_ssd1_v2_x16 -y --brief -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b4_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b3_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b3_v1.vcf.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b4_v1.vcf.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_alldifficultregions.bed.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resourc

### Below is for testing purposes

In [53]:
gene = 'APOB'
df_gene = df.filter(pl.col('gene_name') == gene)
vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
vcf_str = ' '.join(vcf_files)
region_str = ','.join(df_gene['region'].to_list())
mem_level = len(vcf_files)

bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
bcftools_cmd2 = "bcftools view -Ou -T ^" + diff_bed
bcftools_cmd3 = "bcftools norm -Ou -m - -f " + ref_genome
bcftools_cmd4 = "bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT'"
bcftools_cmd5 = "bcftools view -Oz --threads " + str(number_of_threads) + " -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + ".vcf.gz"
bcftools_command = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])

dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'

dx_command = 'dx run swiss-army-knife --instance-type mem' + str(mem_level) + '_ssd1_v2_x4 -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene

print(bcftools_command)
print(dx_command)



bcftools concat -Ou -a -r chr2:21043859-21044078,chr2:21043508-21043556,chr2:21042356-21042481,chr2:21040933-21041088,chr2:21037953-21038116,chr2:21037095-21037260,chr2:21035579-21035713,chr2:21034811-21034906,chr2:21033294-21033523,chr2:21032349-21032586,chr2:21029893-21030020,chr2:21029634-21029790,chr2:21028322-21028543,chr2:21027823-21028070,chr2:21026783-21026969,chr2:21024928-21025129,chr2:21023520-21023697,chr2:21022826-21023047,chr2:21019718-21019910,chr2:21018987-21019118,chr2:21016434-21016654,chr2:21015365-21015550,chr2:21015068-21015265,chr2:21014443-21014598,chr2:21013155-21013538,chr2:21005075-21012656,chr2:21004556-21004680,chr2:21004264-21004457,chr2:21001424-21003339 ukb23157_c2_b3_v1.vcf.gz ukb23157_c2_b4_v1.vcf.gz | bcftools view -Ou -T ^GRCh38_alldifficultregions.bed.gz | bcftools norm -Ou -m - -f GRCh38_reference_genome.fa | bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT' | bcftools view -Oz --threads 4 -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(

In [44]:
!{dx_command}

job-GPG9KY8JqBj5gK412JXv3f8G
